## Instrucciones generales 

1. Forme un grupo de **máximo dos estudiantes**
1. Versione su trabajo usando un **repositorio <font color="red">privado</font> de github**. Agregue a sus compañeros y a su profesor (usuario github: phuijse) en la pestaña *Settings/Manage access*. No se aceptarán consultas si la tarea no está en github. No se evaluarán tareas que no estén en github.
1. Se evaluará el **resultado, la profundidad de su análisis y la calidad/orden de sus códigos** en base al último commit antes de la fecha y hora de entrega". Se bonificará a quienes muestren un método de trabajo incremental y ordenado según el histórico de *commits*
1. Sean honestos, ríganse por el [código de ética de la ACM](https://www.acm.org/about-acm/code-of-ethics-in-spanish)

# ¿Es posible explicar la cantidad de billonarios en base al desarrollo país?  <a class="tocSkip"></a>







En 2006 *Daniel Treisman* publicó un artículo titulado [*Russia Billionaries*](https://pubs.aeaweb.org/doi/pdfplus/10.1257/aer.p20161068) en el cual conectó la cantidad de billonarios de un país con ciertos atributos económicos de los mismos. 

Su conclusión principal fue que Rusia tiene una cantidad de billonarios mayor que la que predicen los indicadores económicos

En esta tarea ustedes analizarán datos macroeconómicos para comprobar o refutar los hallazgos de *D. Treisman*

## Datos

Para esta tarea se les provee de un conjunto de datos `billonarios.csv` indexado por país con los siguientes atributos

- `nbillonarios`: La cantidad de billonarios del pais
- `logpibpc`: El logaritmo del Producto Interno Bruto (PIB) per capita del pais
- `logpob`: El logaritmo de la población del pais
- `gatt`: La cantidad de años que el pais está adherido al *General Agreement on Tariffs and Trade* (GATT)

## Modelo 

El objetivo principal de esta tarea es entrenar un modelo de regresión que prediga la cantidad de billonarios en función de los demás atributos

> El número de billonarios es una variable entera y no-negativa. 

Un modelo apropiado en este caso es la [regresión de Poisson](https://en.wikipedia.org/wiki/Poisson_distribution), donde definimos la probabilidad condicional para un pais $i$ como  

$$
p(y_i | x_i ) = \frac{\lambda_i^{y_i}}{y_i!} \exp \left ({-\lambda_i} \right)
$$

con intensidad

$$
\lambda_i = \exp \left (\theta_0 + \sum_{j=1}^M \theta_j x_{ij} \right)
$$

donde 

- $\theta$ es el vector de parámetros que deseamos ajustar 
- $y_i$ y $x_i$ son la cantidad de billonarios y el vector de atributos del país $i$, respectivamente


$$
\begin{align}
p\left ( y_{i}|x_{i} \right ) &= \frac{\lambda_{i}^{y_{i}}\cdot e^{^{-\lambda_{i}}}}{y_{i}!} \\
\\
\lambda _{i} &= e^{\left ( \theta _{0} + \sum_{j=1}^{M} \theta _{j}\cdot X_{ij} \right )} \\
\\
p\left ( y_{i}|x_{i} \right ) &= \frac{\left (e^{\left ( \theta _{0} + \sum_{j=1}^{M} \theta _{j}\cdot X_{ij} \right )} \right)^{y_{i}}\cdot e^{-e^{\left ( \theta _{0} + \sum_{j=1}^{M} \theta _{j}\cdot X_{ij} \right )} }}{y_{i}!}\\
&=\frac{e^{y_{i}\cdot {\left (\theta_{0} + \sum_{i=1}^N  \theta_{j} \cdot x_{ij}\right)} - e^{{\left (\theta_{0} + \sum_{i=1}^N  \theta_{j} \cdot x_{ij}\right)}}}}{y_{i}!}
\end{align}
$$


## Objetivos

- Implemente la regresion de Poisson usando PyMC
    - Considere un prior normal para los parámetros $\theta$
    - Considere una verosimilitud poisson
- Seleccione dos algoritmo de MCMC para entrenar este modelo. 
    - Muestre y analice las trazas de los parámetros.
    - Muestre y analice el número de muestras efectivo, el estadístico de Gelman-Rubin y la función de autocorrelación
    - En base a las trazas y las métricas calibre los hiperparámetros de su algoritmo para tener la mejro convergencia posible
- Análisis
    - Muestre el posterior de los parámetros obtenidos, su mediana y sus percentiles, ¿Cúales son significativamente distintos de cero?
    - Prediga la cantidad de billonarios usando su modelo y la incertidumbre asociada (posterior predictivo). Muestre graficamente sus resultados
    - ¿Cuáles son los 5 países con mayor error en la predicción? ¿Cuáles países tienen un exceso de billonarios? ¿Cúales paises tienen menos billonarios de lo esperado? ¿Qué puede decir sobre Rusia?

In [1]:
import numpy as np
from scipy.special import factorial
import pandas as pd
import matplotlib.pyplot as plt
from math import e
import pymc3 as pm
import theano.tensor as T
import graphviz

You are running the v4 development version of PyMC3 which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc3/tree/v3


In [2]:
datos = pd.read_csv("billonarios.csv")

In [27]:
datos.head(3)

,pais,nbillonarios,logpibpc,logpob,gatt
0,United States,469,10.786021,19.532846,60
1,Canada,25,10.743365,17.319439,0
2,"Bahamas, The",0,10.072139,12.760934,0


In [4]:
x = datos[['logpibpc', 'logpob', 'gatt']].to_numpy()
y = datos[['nbillonarios']].to_numpy()

In [23]:
theta = np.random.randn(4)
theta[1:]

array([-0.27674243,  0.5000494 ,  1.35968098])

In [6]:
def factorial(y):
    fact = 0
    pos = 0
    array = np.zeros(len(y))
    for i in y:
        suma_log = 0
        for j in range(1, (i[0] + 1)):
            suma_log = suma_log + np.log(j)
        array[pos] = suma_log
        pos = pos + 1
    return (array)

In [ ]:
poisson_pmf = lambda y,theta,x: np.exp(y.T*(theta[0] + np.dot(x, theta[1:])) - np.exp(theta[0] + np.dot(x, theta[1:])) - factorial(y)) 

In [ ]:
with pm.Model() as funcion_poisson:
    # Datos
    #the = np.random.randn(4)
    #media = np.median(theta)
    X_shared = pm.Data("x", x)
    # Priors
    theta0 = pm.Normal('theta0',mu=0,sd=20)
    theta1 = pm.Normal('theta1',mu=0,sd=20)
    theta2 = pm.Normal('theta2',mu=0,sd=20)
    theta3 = pm.Normal('theta3',mu=0,sd=20)
    #pob = pm.Poisson('pob', mu=10, shape=())
    #gatt = pm.Poisson('gatt', mu=10, shape=())
    # Variable determinista
    mu = pm.Deterministic('mu', theta0 + theta*X_shared)
    # Verosimilitud
    Y_obs = pm.Poisson('Y_obs', mu=mu, observed=y)
    

In [7]:
y = y.T

In [31]:
datos['logpibpc']

0      10.786021
1      10.743365
2      10.072139
3      10.223734
4      11.446847
         ...    
192     7.824408
193     8.124372
194    10.356964
195    10.122557
196     7.525887
Name: logpibpc, Length: 197, dtype: float64

In [34]:
datos['logpibpc']

0      10.786021
1      10.743365
2      10.072139
3      10.223734
4      11.446847
         ...    
192     7.824408
193     8.124372
194    10.356964
195    10.122557
196     7.525887
Name: logpibpc, Length: 197, dtype: float64

In [33]:
with pm.Model() as funcion_poisson:

    # define priors, weakly informative Normal
    b0 = pm.Normal("b0", mu=0, sigma=10)
    b1 = pm.Normal("b1", mu=0, sigma=10)
    b2 = pm.Normal("b2", mu=0, sigma=10)
    b3 = pm.Normal("b3", mu=0, sigma=10)

    # define linear model and exp link function
    theta = (
        b0
        + b1 * datos['logpibpc']
        + b2 * datos['logpob']
        + b3 * datos['gatt']
    )
    #print(theta)
    #lam = pm.Deterministic("lam", theta)
    #mu = pm.Deterministic("mu", lam**y/factorial(y)*np.exp(lam))

    ## Define Poisson likelihood
    Y = pm.Poisson("Y", mu=np.exp(theta, observed=datos['pais']))

ValueError: Length of Elemwise{mul,no_inplace}.0 cannot be determined

In [18]:
display(funcion_poisson.free_RVs,
       funcion_poisson.deterministics,
       funcion_poisson.observed_RVs)
display(funcion_poisson)
pm.model_to_graphviz(funcion_poisson)

[b0, b1, b2, b3]

[lam, mu]

[Y]

AttributeError: 'TensorVariable' object has no attribute '__latex__'

ValueError: lam value too large
Apply node that caused the error: poisson_rv{0, (0,), int64, True}(RandomStateSharedVariable(<RandomState(MT19937) at 0x7FC1EF92D240>), TensorConstant{[]}, TensorConstant{4}, mu)
Toposort index: 10
Inputs types: [RandomStateType, TensorType(int64, vector), TensorType(int64, scalar), TensorType(float64, row)]
Inputs shapes: ['No shapes', (0,), (), (1, 197)]
Inputs strides: ['No strides', (8,), (), (1576, 8)]
Inputs values: [RandomState(MT19937) at 0x7FC1EF92D240, array([], dtype=int64), array(4), 'not shown']
Outputs clients: [['output'], []]

HINT: Re-running with most Aesara optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the Aesara flag 'optimizer=fast_compile'. If that does not work, Aesara optimizations can be disabled with 'optimizer=None'.
HINT: Use the Aesara flag `exception_verbosity=high` for a debug print-out and storage map footprint of this Apply node.

In [ ]:
x_test = np.linspace(-5, 5, num=100)

with funcion_poisson:
    prior_checks = pm.sample_prior_predictive(samples=50, var_names=['pib', 'pob', 'gatt'])
    
fig, ax = plt.subplot(figsize=(6,3) tight_layout=True)
ax.set_xlabel('X')
ax.set_ylaber('Y')

for pib, pob, gatt in zip(prior_checks['pib'], prior_checks['pob'], prior_checks['gatt']):
    ax.plot(x_test, '''x_test en funcion poisson?''')

In [ ]:
x[:,0]